In [ ]:
# Import required libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Initialize Spark Session with BigQuery connector
spark = SparkSession.builder \
    .appName("NYC_Route_Analysis") \
    .config("spark.jars.packages", "com.google.cloud.spark:spark-bigquery-with-dependencies_2.12:0.32.0") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

# Set your GCP project
PROJECT_ID = "dtc-de-course-457315"
DATASET_ID = "nyc_taxi_data"

print(f"Spark Version: {spark.version}")
print(f"Analyzing data from: {PROJECT_ID}.{DATASET_ID}")


In [ ]:
# Load taxi trip data from BigQuery
yellow_taxi_df = spark.read \
    .format("bigquery") \
    .option("table", f"{PROJECT_ID}.{DATASET_ID}.yellow_taxi_external_table") \
    .load()

# Load zone lookup data
zone_df = spark.read \
    .format("bigquery") \
    .option("table", f"{PROJECT_ID}.{DATASET_ID}.taxi_zone_external_table") \
    .load()

print(f"Yellow Taxi Records: {yellow_taxi_df.count():,}")
print(f"Zone Records: {zone_df.count():,}")

# Clean and preprocess the data
cleaned_trips = yellow_taxi_df \
    .filter(
        (col("trip_distance") > 0) & 
        (col("trip_distance") < 100) &  # Remove unrealistic distances
        (col("tpep_pickup_datetime") < col("tpep_dropoff_datetime")) &
        (col("fare_amount") > 0) &
        (col("PULocationID").isNotNull()) &
        (col("DOLocationID").isNotNull())
    ) \
    .withColumn(
        "trip_duration_minutes", 
        (unix_timestamp(col("tpep_dropoff_datetime")) - unix_timestamp(col("tpep_pickup_datetime"))) / 60
    ) \
    .filter(
        (col("trip_duration_minutes") >= 1) & 
        (col("trip_duration_minutes") <= 120)  # 1 min to 2 hours
    ) \
    .withColumn(
        "avg_speed_mph",
        col("trip_distance") / (col("trip_duration_minutes") / 60)
    ) \
    .filter(col("avg_speed_mph") <= 80)  # Remove unrealistic speeds

print(f"Cleaned Records: {cleaned_trips.count():,}")


In [ ]:
# Join with zone data to get location names
pickup_zones = zone_df.select(
    col("LocationID").alias("pickup_zone_id"),
    col("Zone").alias("pickup_zone_name"),
    col("Borough").alias("pickup_borough")
)

dropoff_zones = zone_df.select(
    col("LocationID").alias("dropoff_zone_id"),
    col("Zone").alias("dropoff_zone_name"),
    col("Borough").alias("dropoff_borough")
)

# Create comprehensive route analysis with zone information
route_analysis = cleaned_trips \
    .join(pickup_zones, col("PULocationID") == col("pickup_zone_id"), "left") \
    .join(dropoff_zones, col("DOLocationID") == col("dropoff_zone_id"), "left") \
    .withColumn("pickup_hour", hour(col("tpep_pickup_datetime"))) \
    .withColumn("day_of_week", dayofweek(col("tpep_pickup_datetime"))) \
    .withColumn("route_id", concat(col("PULocationID"), lit("-"), col("DOLocationID"))) \
    .withColumn("route_description", concat(col("pickup_zone_name"), lit(" → "), col("dropoff_zone_name")))

route_analysis.cache()  # Cache for multiple operations
print("Route analysis dataset created and cached")

# ===============================
# CORE SECTION 6.2 ANALYSIS: FASTEST ROUTES
# ===============================

# Aggregate route statistics (similar to typical Kaggle analysis)
route_stats = route_analysis.groupBy(
    "route_id", "pickup_zone_name", "dropoff_zone_name", 
    "pickup_borough", "dropoff_borough", "route_description"
).agg(
    count("*").alias("trip_count"),
    avg("trip_duration_minutes").alias("avg_duration_minutes"),
    min("trip_duration_minutes").alias("fastest_duration_minutes"),
    avg("avg_speed_mph").alias("avg_speed_mph"),
    max("avg_speed_mph").alias("max_speed_mph"),
    avg("trip_distance").alias("avg_distance_miles"),
    avg("fare_amount").alias("avg_fare"),
    stddev("avg_speed_mph").alias("speed_std_dev")
).filter(col("trip_count") >= 100)  # Ensure statistical significance

print(f"Route combinations with 100+ trips: {route_stats.count():,}")

# Find fastest routes by average speed
fastest_routes_by_speed = route_stats.orderBy(col("avg_speed_mph").desc()).limit(20)

print("\n🚀 TOP 20 FASTEST ROUTES BY AVERAGE SPEED:")
fastest_routes_by_speed.select(
    "route_description", 
    round("avg_speed_mph", 2).alias("avg_speed_mph"), 
    round("avg_duration_minutes", 2).alias("avg_duration_min"), 
    round("avg_distance_miles", 2).alias("avg_distance_mi"), 
    "trip_count"
).show(20, truncate=False)
